In [ ]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
from tqdm import tqdm
import os

In [ ]:
# pc端的user-agent
user_agent_pc = [
    # 谷歌
    'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.71 Safari/537.36',
    'Mozilla/5.0.html (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.html.1271.64 Safari/537.11',
    'Mozilla/5.0.html (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.html.648.133 Safari/534.16',
    # 火狐
    'Mozilla/5.0.html (Windows NT 6.1; WOW64; rv:34.0.html) Gecko/20100101 Firefox/34.0.html',
    'Mozilla/5.0.html (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
    # opera
    'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.html.2171.95 Safari/537.36 OPR/26.0.html.1656.60',
    # qq浏览器
    'Mozilla/5.0.html (compatible; MSIE 9.0.html; Windows NT 6.1; WOW64; Trident/5.0.html; SLCC2; .NET CLR 2.0.html.50727; .NET CLR 3.5.30729; .NET CLR 3.0.html.30729; Media Center PC 6.0.html; .NET4.0C; .NET4.0E; QQBrowser/7.0.html.3698.400)',
    # 搜狗浏览器
    'Mozilla/5.0.html (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.html.963.84 Safari/535.11 SE 2.X MetaSr 1.0.html',
    # 360浏览器
    'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.html.1599.101 Safari/537.36',
    'Mozilla/5.0.html (Windows NT 6.1; WOW64; Trident/7.0.html; rv:11.0.html) like Gecko',
    # uc浏览器
    'Mozilla/5.0.html (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.html.2125.122 UBrowser/4.0.html.3214.0.html Safari/537.36',
]
# 移动端的user-agent
user_agent_phone = [
    # IPhone
    'Mozilla/5.0.html (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.html.2 Mobile/8J2 Safari/6533.18.5',
    # IPAD
    'Mozilla/5.0.html (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.html.2 Mobile/8C148 Safari/6533.18.5',
    'Mozilla/5.0.html (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.html.2 Mobile/8J2 Safari/6533.18.5',
    # Android
    'Mozilla/5.0.html (Linux; U; Android 2.2.1; zh-cn; HTC_Wildfire_A3333 Build/FRG83D) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0.html Mobile Safari/533.1',
    'Mozilla/5.0.html (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0.html Mobile Safari/533.1',
    # QQ浏览器 Android版本
    'MQQBrowser/26 Mozilla/5.0.html (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0.html Mobile Safari/533.1',
    # Android Opera Mobile
    'Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10',
    # Android Pad Moto Xoom
    'Mozilla/5.0.html (Linux; U; Android 3.0.html; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0.html Safari/534.13',
]
 
def get_user_agent_pc():
    return random.choice(user_agent_pc)
 
def get_user_agent_phone():
    return random.choice(user_agent_phone)

In [ ]:
def get_file(url,file_name,path):
    
    if os.path.exists(path+file_name):
        print('已存在pdf，自动跳过！')
        return
    
    if '招商银行' in file_name or '招银' in file_name:
        pass
    else:
        print('{} 不属于招商银行的理财产品，跳过！'.format(file_name))
        return 
    num_retries=1
    while num_retries<=100:
        try:
            u = urllib.request.urlopen(url)
        except urllib.URLError as e:
            print('Download error:' % e.reason)
            print('正在尝试第{}次连接'.format(num_retries))
            num_retries += 1
            sleep(1)
        else:
            break
            
    f = open(path+file_name, 'wb')
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break
        f.write(buffer)
    print('正在写入 —— {}'.format(file_name))
    f.close()

In [ ]:
def point_to_page(browser,page):
    browser.find_element_by_xpath('/html/body/form/div[5]/div[2]/div[10]/div[5]/div/div/div/input').send_keys(str(page))
    browser.find_element_by_xpath('/html/body/form/div[5]/div[2]/div[10]/div[5]/div/div/a[7]').click()

In [ ]:
def get_pdf(page,pageTo=0):
    title = 'http://www.cmbchina.com'
    browser = webdriver.Chrome('./chromedriver.exe') 
    # 设置窗口大小
    browser.set_window_size(1800, 1000)
    browser.get('http://www.cmbchina.com/cfweb/personal/default.aspx')
    for i in tqdm(range(page)):
        # 向下滚动5次，每次200个像素
        for j in range(5):
            browser.execute_script('window.scrollBy(0,10000)')
            sleep(0.5)
            
        if pageTo != 0:
            point_to_page(browser,pageTo)
            pageTo = 0 
            continue
            
        html = browser.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(html)
        l = soup.find('div',class_='c_list')
        details = []
        
        while len(l.find_all('div',class_='prdBlock')) < 70:
            # 向下滚动5次，每次200个像素
            for k in range(5):
                browser.execute_script('window.scrollBy(0,10000)')
                sleep(0.5)
            html = browser.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(html)
            l = soup.find('div',class_='c_list')
            print('页面未拉到底，再次进行下拉！')
            
        for div in l.find_all('div',class_='prdBlock'):
            details.append(div.find('table',class_='buyArea').find('a')['href'])
        
        for detail in details:
            detail_html =  title + detail
            for name in ['&type=prodexplain','&type=prodrisk']:
                if random.random() < 0.5:
                    headers={'User-Agent':get_user_agent_pc()}
                else:
                    headers={'User-Agent':get_user_agent_phone()}
                res = requests.get(
                    detail_html + name, 
                    headers=headers
                ).text
                html = BeautifulSoup(res).find('div',class_='m_finance_container_p').find('a')
                # 如果指定位置不存在pdf，则跳过
                if html is not None:
                    file_name = html.getText()
                    url = html['href']
                    if name[-1] == 'n':
                        get_file(url,file_name,'./说明书pdf/')
                    else:
                        get_file(url,file_name,'./风险揭示书pdf/')
                else:
                    print('指定位置不存在pdf，跳过！')
            sleep(random.random())
            
        if i == 0:
            browser.find_element_by_xpath('/html/body/form/div[5]/div[2]/div[10]/div[5]/div/div/a[5]').click()
        else:
            browser.find_element_by_xpath('/html/body/form/div[5]/div[2]/div[10]/div[5]/div/div/a[7]').click()
                
    browser.close()

In [ ]:
get_pdf(11,6)